In [84]:
import tqdm
import torch, cv2
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torchvision.ops import box_iou
import sys
sys.path.append('../scripts/')
from resize_boxes import resize_values
from tqdm import tqdm
import time
import pickle

In [19]:
image_paths = np.loadtxt('../../data/CUB_200_2011/images.txt', dtype=str) # all image paths
bboxes_file_path = "../../data/CUB_200_2011/bounding_boxes.txt"
bboxes = np.loadtxt(bboxes_file_path, dtype=int, delimiter=" ", ndmin=2)[:, 1:] # Take only x, y, w, h 
# bboxes[:, 2] = bboxes[:, 2] + bboxes[:, 0] # get corner point
# bboxes[:, 3] = bboxes[:, 3] + bboxes[:, 1] # get corner point

<ipython-input-19-3b23cd042992>:3: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  bboxes = np.loadtxt(bboxes_file_path, dtype=int, delimiter=" ", ndmin=2)[:, 1:] # Take only x, y, w, h


In [30]:
updated_boxes = []
for x in tqdm(range(len(image_paths))):
    # print('../../data/CUB_200_2011/images/'+ image_paths[x][1])
    img = np.array(PIL.Image.open('../../data/CUB_200_2011/images/'+ image_paths[x][1]))
    updated_boxes.append(resize_values(bboxes[x], img.shape, (416, 416), return_old_box=False))

100%|██████████| 11788/11788 [00:22<00:00, 530.91it/s]


In [31]:
updated_boxes[0]

[49.92, 33.528358208955225, 220.48000000000002, 343.9761194029851]

In [32]:
updated_boxes = torch.Tensor(updated_boxes)

In [70]:
def IoU(clusters: torch.tensor, bboxes: torch.tensor):
    iou_values = box_iou(clusters, bboxes)
    return iou_values

In [87]:
def KMeans(bboxes:torch.tensor, k:int, dist=torch.mean, stop_iter=4):
    rows = bboxes.shape[0]
    distances = torch.empty((rows, k))
    last_clusters = torch.zeros((rows, ))

#   We will randomly pick them and tell them they are the cluster centroids.....
    cluster_indxs = np.random.choice(rows, k, replace=False) # choose unique indexs in rows
    clusters = bboxes[cluster_indxs].clone()

    iteration = 0
    while True:
        # calculate the distances 
        distances = IoU(bboxes, clusters)

        nearest_clusters = torch.argmax(distances, dim=1) # 0, 1, 2 ... K   

        if (last_clusters == nearest_clusters).all(): # break if nothing changes
            iteration += 1
            if iteration == stop_iter:
                break
        else:
            iteration = 0
        # Take the mean and step for cluster coordiantes 
        for cluster in range(k):
            clusters[cluster] = torch.mean(bboxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters.clone()
    return clusters, distances

In [88]:
clusterList = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
mean_distances, anchorBoxes = [], []

for cluster_k in tqdm(clusterList):
    anchors, distances = KMeans(updated_boxes, k=cluster_k)
    indxs = torch.argmax(distances, dim=1)
    filtered_distances = []
    for i, distance in enumerate(distances):
        filtered_distances.append(distance[indxs[i]].item())
    mean_distances.append(np.mean(filtered_distances))
    anchorBoxes.append(anchors)

 16%|█▌        | 3/19 [9:06:00<48:32:04, 10920.30s/it]


KeyboardInterrupt: 

In [ ]:
plt.plot(clusterList, mean_distances)
plt.scatter(clusterList, mean_distances)
plt.title("Mean IoU Score")
plt.xlabel("Number of clusters")
plt.ylabel("IoU score")

In [ ]:
with open('./mean_dist.pkl', 'wb') as handle:
    pickle.dump(mean_distances, handle)
with open('./anchor_boxes.pkl', 'wb') as handle:
    pickle.dump(anchorBoxes, handle)